In [1]:
import pyconll, keras, pickle, os, random, nltk, datetime, warnings, gc, urllib.request, zipfile
import numpy as np
import math
from scipy.sparse import hstack, vstack
import matplotlib.pyplot as plt
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import FastText
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, GridSearchCV, learning_curve, cross_val_score
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, BatchNormalization, Dropout, Input, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import np_utils
# from sklearn.metrics.classification import UndefinedMetricWarning
# warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)
from numpy.random import seed

2023-03-09 09:44:33.631431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 09:44:33.908296: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-09 09:44:33.908592: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-09 09:44:35.671632: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
ROOT_DIR = os.path.dirname("/home/chitrang/Documents/CSE-582/") # setting the root dir
POS_DIR = os.path.join(ROOT_DIR, 'dataset')
pos_train = os.path.join(POS_DIR, "train.txt")

In [3]:
def format_data(fname, include_y=True):
    sentences = [] # master list
    with open(fname) as f:
        content = f.readlines()
    
    sentence = [] # local list
    for line in content:
        if line !='\n':
            line = line.strip() # remove leading/trailing spaces
            word = line.split()[0].lower() # get the word
            if include_y:
                pos = ""
                pos = line.split()[1] # get the pos tag
                sentence.append((word, pos)) # create a pair and save to local list
            else:
                sentence.append(word)
        else:
            sentences.append(sentence) # once a \n is detected, append the local sentence to master sentence
            sentence = []
    return sentences

In [4]:
train_sentences = format_data(pos_train)

In [5]:
print("Tagged sentences in train set: ", len(train_sentences))
print("Tagged words in train set:", len([item for sublist in train_sentences for item in sublist]))

Tagged sentences in train set:  8936
Tagged words in train set: 211727


In [6]:
# # Dataset Split
# total_train = len(train_sentences)
# print(total_train)
# val_split = math.floor(0.2 * total_train)
# print(val_split)
# train_sentences, val_sentences = train_sentences[:(total_train-val_split)], train_sentences[(total_train-val_split):]
# print(len(train_sentences), len(val_sentences))
# print(len(train_sentences)+len(val_sentences))

In [7]:
# print(train_sentences[0])

In [8]:
def tag_sequence(sentences):
    return [[t for w, t in sentence] for sentence in sentences]

def text_sequence(sentences):
    return [[w for w, t in sentence] for sentence in sentences]

def id2word(sentences):
    wordlist = [item for sublist in text_sequence(sentences) for item in sublist]
    id2word = {k:v for k,v in enumerate(wordlist)}
    return id2word

def untag(tagged_sentence):
    return [w for w, _ in tagged_sentence]

def untag_pos(tagged_sentence):
    return [t for _, t in tagged_sentence]

def build_vocab(sentences):
    vocab =set()
    for sentence in sentences:
        for word in untag(sentence):
            vocab.add(word)
    return sorted(list(vocab))

In [9]:
# vocab = build_vocab(train_sentences)

In [10]:
# len(vocab)

In [11]:
embs_path = '/home/chitrang/Downloads/wiki-news-300d-1M.vec'
embeddings = KeyedVectors.load_word2vec_format(embs_path, binary=False)

In [12]:
w2c = dict()
for item in embeddings.key_to_index:
    w2c[item] = embeddings.key_to_index[item]

In [13]:
dim = embeddings.vectors.shape[1]
pad = np.zeros(dim)
np.random.seed(3)
oov = np.random.uniform(-0.25, 0.25, dim)

In [65]:
def features_embs(sentence, index, window=1):
    unknown=0
    vec = np.array([])
    for i in range(index-window, index+window+1):
#         if i < 0:
#             vec = np.append(vec, pad)
#         if i > len(sentence)-1:
#             vec = np.append(vec, pad)
        try:
            vec = np.append(vec, embeddings[sentence[i]])
        except:
            vec = np.append(vec, oov)
            unknown += 1
    return vec, unknown

In [53]:
def features_basic(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'nb_terms': len(sentence),        
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'i-1_prefix-3': '' if index == 0 else sentence[index-1][:3],        
        'i-1_suffix-3': '' if index == 0 else sentence[index-1][-3:],        
        'i+1_prefix-3': '' if index == len(sentence) - 1 else sentence[index+1][:3],        
        'i+1_suffix-3': '' if index == len(sentence) - 1 else sentence[index+1][-3:],        
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:],
    }

In [54]:
def transform_to_dataset(tagged_sentences, window):
    i=0
    X, y = [], []
    for doc_index, tagged in enumerate(tagged_sentences):
        for index in range(len(tagged)):
            X.append([features_basic(untag(tagged), index),\
                      features_embs(untag(tagged), index, window)[0],\
                     ])
            y.append(tagged[index][1])
            #features_embs(untag(tagged), index, window)[1]
            k = features_embs(untag(tagged), index, window)[1]
            i += k
    return X, y, i

In [17]:
def transform_test_sentence(sentence, window):
    X = []
    for index in range(len(sentence)):
            X.append([
                      features_basic(sentence, index),\
                      features_embs(sentence, index, window),\
                     ])
    return X

In [47]:
def transform_to_dataset_unknown(sentences, window):
    X = []
    for doc_index, sentence in enumerate(sentences):
        for index in range(len(sentence)):
            X.append([features_basic(sentence, index),\
                      features_embs(sentence, index, window),\
                     ])
    return X

In [18]:
# print(features_embs(train_sentences[0], 0))

In [19]:
# len(untag_train)

In [20]:
# untag_sentence_vector = []
# for i, word in enumerate(untag_train):
#     print(len(features_embs(untag_train, i, window=1)[0]))

In [21]:
# type(train_sentences)

In [22]:
# temp_list = []
# temp_list.append(train_sentences[0])
# temp_list.append(train_sentences[1])

In [23]:
# 2 sentences, first one with 37 words, second one with 27 words

In [24]:
# X, y, i = transform_to_dataset(temp_list, 1)
# # transformed = transform_to_dataset(temp_list, 1)

In [25]:
# transformed[0] -> untagged part, transformed[1] -> pos_tags
# transformed[0][0] -> each sentence
# transformed[0][0][0] -> features basic list
# transformed[0][0][1] -> features embs

In [26]:
# len(transformed[1]) -> 64 i.e 37+27

In [27]:
# X = transformed[0][0] -> for each sentence, for each word it has features basic, features embs

In [28]:
# y = transformed[1] -> all pos tags collected in a single list for the whole dataset

In [62]:
from sklearn.feature_extraction import FeatureHasher, DictVectorizer
# classical
def vectorize(train, window=1):
    print('Embeddings window method')
    print('Vectorizing Dataset...')
    X_train, y_train, unk_tr = transform_to_dataset(train, window=window)
    X_train = [x[1] for x in X_train]
    X_train = np.asarray(X_train)
#     print('Vectorizing train...')
#     X_train, y_train, _ = transform_to_dataset(train, window=window)
# #     hasher = FeatureHasher(n_features=2**20)
# #     X_train = hasher.transform([x[0] for x in X_train])
#     v = DictVectorizer(sparse=True) 
#     X_train = v.fit_transform([x[0] for x in X_train])
    
# #     print('Vectorizing val...')
# #     X_val, y_val, _ = transform_to_dataset(val, window=window)
# #     X_val = v.fit_transform([x[0] for x in X_val])
# #     print('Dataset vectorized.')
# #     print('Train shape:', X_train.shape)
# #     print('Val shape:', X_val.shape)
#     X_train = X_train
#     X_val = X_val
    return X_train, y_train

In [48]:
from sklearn.feature_extraction import FeatureHasher, DictVectorizer
def vectorize_unknown(test, window=1):
    print('Vectorizing test...')
    X_test = transform_to_dataset_unknown(test, window=window)
#     hasher = FeatureHasher(n_features=2**20)
#     X_train = hasher.transform([x[0] for x in X_train])
    v = DictVectorizer(sparse=True) 
    X_test = v.fit_transform([x[0] for x in X_test])
    X_test = X_test
    return X_test

In [66]:
X_train, y_train = vectorize(train_sentences)

Embeddings window method
Vectorizing Dataset...


In [67]:
X_train.shape

(211727, 900)

In [68]:
classes = sorted(list(set(y_train)))

In [70]:
X_train.shape

(211727, 900)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [71]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
# y_val = le.transform(y_val)
y_train = keras.utils.to_categorical(y_train)
# y_val = keras.utils.to_categorical(y_val)

print(y_train.shape)

(211727, 44)


In [35]:
# model = Sequential()
# # In the first layer, we specify the input data shape

# model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(y_train.shape[1], activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [36]:
# nb_epoch = 10
# batch_size = 128
# early_stopping = EarlyStopping(monitor = 'val_acc', patience = 5)
# history = model.fit(X_train, y_train,
#                     epochs=nb_epoch,
#                     batch_size=batch_size,
#                     shuffle=True,
#                     validation_data=(X_val, y_val),
#                     verbose=1,
#                     callbacks=[early_stopping])

In [37]:
X_full_train = vstack((X_train, X_val)).tocsr()
y_full_train = np.append(y_train, y_val, axis=0)

In [38]:
X_train.shape

(169381, 70531)

In [130]:
model = Sequential()
# Dense(200) is a fully-connected layer with 200 hidden units.
# In the first layer, we specify the input data shape

model.add(Dense(200, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 200)               180200    
                                                                 
 dropout_6 (Dropout)         (None, 200)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                12864     
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 44)                2860      
                                                                 
Total params: 195,924
Trainable params: 195,924
Non-trainable params: 0
_________________________________________________________________


In [131]:
model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,
                    verbose=1,
                   ) 

Epoch 1/20
6617/6617 [==============================] - 20s 3ms/step - loss: 0.5049 - accuracy: 0.8575
Epoch 2/20
6617/6617 [==============================] - 20s 3ms/step - loss: 0.2671 - accuracy: 0.9200
Epoch 3/20
6617/6617 [==============================] - 19s 3ms/step - loss: 0.2341 - accuracy: 0.9285
Epoch 4/20
6617/6617 [==============================] - 20s 3ms/step - loss: 0.2166 - accuracy: 0.9335
Epoch 5/20
6617/6617 [==============================] - 20s 3ms/step - loss: 0.2018 - accuracy: 0.9378
Epoch 6/20
6617/6617 [==============================] - 19s 3ms/step - loss: 0.1939 - accuracy: 0.9401
Epoch 7/20
6617/6617 [==============================] - 19s 3ms/step - loss: 0.1878 - accuracy: 0.9409
Epoch 8/20
6617/6617 [==============================] - 20s 3ms/step - loss: 0.1819 - accuracy: 0.9428
Epoch 9/20
6617/6617 [==============================] - 20s 3ms/step - loss: 0.1763 - accuracy: 0.9451
Epoch 10/20
6617/6617 [==============================] - 18s 3ms/step - l

In [142]:
# model.save("my_model.h5")

In [74]:
X_train.shape

(211727, 900)

In [75]:
y_train.shape

(211727, 44)

In [148]:
# train_sentences

In [76]:
pos_test = os.path.join(POS_DIR, "test.txt")

In [77]:
test_sentences = format_data(pos_test, False)

In [79]:
# X_test_ = vectorize_unknown(test_sentences)

In [80]:
# X_test_

In [81]:
lets_test = transform_test_sentence(test_sentences[0], 1)

In [82]:
dim = embeddings.vectors.shape[1]
pad = np.zeros(dim)

In [83]:
lets_test[1][1][0].shape

(900,)

In [84]:
# v = DictVectorizer(sparse=True) 
# X_embs = v.fit_transform([x[0] for x in lets_test])
# X_embs = X_embs
# lets_test = X_embs
X_embs = [x[1][0] for x in lets_test]
X_embs = np.asarray(X_embs)
lets_test = X_embs

In [132]:
type(lets_test)

numpy.ndarray

In [158]:
predict_x=model.predict(lets_test, batch_size=1) 
classes_x=np.argmax(predict_x,axis=1)

28/28 [==============================] - 0s 1ms/step


In [159]:
argmaxed = np.argmax(predict_x, axis=1)

In [161]:
predict_x

(28, 44)

In [123]:
print('Here is what our MLP tagger predicts for the test sentence:\n',list(zip(test_sentences[0], le.inverse_transform(argmaxed))))

Here is what our MLP tagger predicts for the test sentence:
 [('rockwell', 'NNP'), ('international', 'NNP'), ('corp.', 'NNP'), ("'s", 'DT'), ('tulsa', 'NNP'), ('unit', 'NNP'), ('said', 'VBD'), ('it', 'PRP'), ('signed', 'VBD'), ('a', 'DT'), ('tentative', 'JJ'), ('agreement', 'NN'), ('extending', 'VBG'), ('its', 'PRP$'), ('contract', 'NN'), ('with', 'IN'), ('boeing', 'NNP'), ('co.', 'NNP'), ('to', 'TO'), ('provide', 'VB'), ('structural', 'JJ'), ('parts', 'NNS'), ('for', 'IN'), ('boeing', 'NNP'), ("'s", 'POS'), ('747', 'NN'), ('jetliners', 'NNS'), ('.', '.')]


In [98]:
pos_test_correct = '/home/chitrang/Downloads/test.txt'

In [99]:
correct_test_sen = format_data(pos_test_correct)

In [144]:
predicted_sen = zip(test_sentences[0], le.inverse_transform(argmaxed))

In [145]:
predicted_sen

In [125]:
correct_test_sen[0]

[('rockwell', 'NNP'),
 ('international', 'NNP'),
 ('corp.', 'NNP'),
 ("'s", 'POS'),
 ('tulsa', 'NNP'),
 ('unit', 'NN'),
 ('said', 'VBD'),
 ('it', 'PRP'),
 ('signed', 'VBD'),
 ('a', 'DT'),
 ('tentative', 'JJ'),
 ('agreement', 'NN'),
 ('extending', 'VBG'),
 ('its', 'PRP$'),
 ('contract', 'NN'),
 ('with', 'IN'),
 ('boeing', 'NNP'),
 ('co.', 'NNP'),
 ('to', 'TO'),
 ('provide', 'VB'),
 ('structural', 'JJ'),
 ('parts', 'NNS'),
 ('for', 'IN'),
 ('boeing', 'NNP'),
 ("'s", 'POS'),
 ('747', 'CD'),
 ('jetliners', 'NNS'),
 ('.', '.')]

In [126]:
def compare_results(predicted_sen, correct_sen, acc = True):
    total = 0
    correct_total = 0
    if acc:
        for predict, correct in zip(predicted_sen, correct_sen):
                if predict[1] == correct[1]:
                    correct_total = correct_total + 1
                total = total + 1
        acc = correct_total / total
        return acc
    else:
        for predict, correct in zip(predicted_sen, correct_sen):
                if predict[1] == correct[1]:
                    print(f"{predict[1]} \t {correct[1]}")
                else:
                    print(f"{predict[1]} \t {correct[1]} <------ Error")

In [127]:
compare_results(predicted_sen, correct_test_sen[0], False)

NNP 	 NNP
NNP 	 NNP
NNP 	 NNP
DT 	 POS <------ Error
NNP 	 NNP
NNP 	 NN <------ Error
VBD 	 VBD
PRP 	 PRP
VBD 	 VBD
DT 	 DT
JJ 	 JJ
NN 	 NN
VBG 	 VBG
PRP$ 	 PRP$
NN 	 NN
IN 	 IN
NNP 	 NNP
NNP 	 NNP
TO 	 TO
VB 	 VB
JJ 	 JJ
NNS 	 NNS
IN 	 IN
NNP 	 NNP
POS 	 POS
NN 	 CD <------ Error
NNS 	 NNS
. 	 .


In [164]:
test_sentences = format_data(pos_test, False)
preprocessed_test_data = []
def embed_test_sentences(sentence):
    X_embs = [x[1][0] for x in sentence]
    X_embs = np.asarray(X_embs)
    return X_embs

def preprocess_unlabelled_test_data(test_sentences):
    for sentence in test_sentences:
        sentence = transform_test_sentence(sentence, 1)
        embedded = embed_test_sentences(sentence)
        preprocessed_test_data.append(embedded)


In [165]:
preprocess_unlabelled_test_data(test_sentences)

In [166]:
predicted_data = []
arg_max_dict = []
def test_set_predictions(preprocessed_test_data, test_sentences):
    for sentence in preprocessed_test_data:
        predict_x=model.predict(sentence, batch_size=1, verbose=0) 
        predict_x = np.argmax(predict_x, axis=1)
        arg_max_dict.append(predict_x)
        
    for index in range(len(test_sentences)):
        predicted_sen = list(zip(test_sentences[index], le.inverse_transform(arg_max_dict[index])))
        predicted_data.append(predicted_sen)


In [168]:
test_set_predictions(preprocessed_test_data, test_sentences)

In [171]:
def compare_with_test_set(correct_set):
    total = 0
    correct = 0
    for predicted_sentence, correct_sentence in zip(predicted_data, correct_set):
        for predicted_word, correct_word in zip(predicted_sentence, correct_sentence):
            total = total + 1
            if predicted_word[1] == correct_word[1]:
                correct = correct + 1
    
    accuracy = (correct / total) * 100
    return accuracy

In [172]:
compare_with_test_set(correct_test_sen)

95.45771154779746

In [183]:
labelled_data = []
def generate_labelled_from_unlabelled(predicted_data, correct_sen):
    for predicted_sentence, correct_sentence in zip(predicted_data, correct_sen):
        for predicted_word, correct_word in zip(predicted_sentence, correct_sentence):
            labelled_data.append((predicted_word[0], predicted_word[1]))

In [184]:
generate_labelled_from_unlabelled(predicted_data, correct_test_sen)

In [191]:
len([item for sublist in test_sentences for item in sublist])

47377